<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# installs
!pip install jupyter-dash
!pip install dash-cytoscape

  Using cached jupyter_dash-0.4.0-py3-none-any.whl (20 kB)
Processing /home/assaf127/.cache/pip/wheels/e3/1e/fa/0c810b67f082a7b9f33a128f6374b03cf5245edc45e37adbf8/dash-1.20.0-py3-none-any.whl
Processing /home/assaf127/.cache/pip/wheels/ac/7e/fd/807844722d79d8babcd27b16e5f7ecc7b476c45ca607c11729/dash_renderer-1.9.1-py3-none-any.whl
Processing /home/assaf127/.cache/pip/wheels/0c/9c/bd/06904831bdaa998ad1f53584553c5e06324814328666699128/dash_table-4.11.3-py3-none-any.whl
Processing /home/assaf127/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
Processing /home/assaf127/.cache/pip/wheels/2a/a3/33/bf6842d5d279588f8989ee8c2f9d9d8feb7af82216493a1eea/dash_html_components-1.1.3-py3-none-any.whl
Processing /home/assaf127/.cache/pip/wheels/68/92/0c/4ac6d91639a754ebf3adfab7172c61fe408e7c71d00b8cbf66/dash_core_components-1.16.0-py3-none-any.whl
  Using cached dash_cytoscape-0.2.0.tar.gz (3.6 MB)
  Created wheel for dash-cytoscape: filename

In [3]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json
import statistics

In [4]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())

In [5]:
graph = {node["data"]["id"]: {"neighbors": list(), "inv": node["classes"].startswith("inv"), "origin": int(node["classes"].split(" ")[-1])} for node in data["nodes"]}
for edge in data["edges"]:
  graph[edge["data"]["source"]]["neighbors"].append(edge["data"]["target"])
degrees = []
for node_id, node_content in graph.items():
  node_content['degree'] = len(node_content["neighbors"])
  degrees.append(node_content['degree'])
std_deg, mean_deg, median_deg, max_deg = statistics.stdev(degrees), statistics.mean(degrees), statistics.median(degrees), max(degrees)
high_deg_nodes = [node_id for node_id, node_content in graph.items() if node_content['degree'] > mean_deg + std_deg]
high_deg_nodes[-10: -1]

['2075', '2088', '1991', '2050', '2129', '2003', '1984', '2109', '2080']

In [6]:
invariants = [node_id for node_id, node_content in graph.items() if node_content["inv"]]
len(invariants) / len(graph)

0.4405430711610487

In [7]:
group0 = [node_id for node_id, node_content in graph.items() if node_content["origin"] == 0]
len(group0) / len(graph)

0.9948501872659176

0 - reduceGeneralizeReason
1 - reducePushClauseReason
2 - predReason
3- noReason

In [9]:
# Build App
app = JupyterDash(__name__)
elements = data["nodes"] + data["edges"]

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '100%', 'height': '1000px'},
                                      stylesheet = [
                                                      # Group selectors
                                                      {
                                                          'selector': 'node',
                                                          'style': {
                                                              'label': 'data(count)'
                                                          }
                                                      },
                                                      {
                                                          'selector': 'edge',
                                                          'style': {
                                                              'target-arrow-color': '#ccc',
                                                              'target-arrow-shape': 'triangle'
                                                          }
                                                      },
                                                      # Class selectors
                                                      {
                                                          'selector': '.inv',
                                                          'style': {
                                                              'shape': 'circle'
                                                          }
                                                      },
                                                      {
                                                          'selector': '.non_inv',
                                                          'style': {
                                                              'shape': 'rectangle'
                                                          }
                                                      },
                                                      {
                                                          'selector': '.0',
                                                          'style': {
                                                              'background-color': 'blue'
                                                          }
                                                      },
                                                      {
                                                          'selector': '.1',
                                                          'style': {
                                                              'background-color': 'yellow'
                                                          }
                                                      },
                                                      {
                                                          'selector': '.2',
                                                          'style': {
                                                              'background-color': 'green'
                                                          }
                                                      },
                                                      {
                                                          'selector': '.3',
                                                          'style': {
                                                              'background-color': 'purple'
                                                          }
                                                      }
                                                  ]
                                      ),
                       html.Pre(id='cytoscape-tapNodeData-json', style=styles['pre'])
                       ])

@app.callback(Output('cytoscape-tapNodeData-json', 'children'),
              Input('cytoscape-event-callbacks-1', 'tapNodeData'))
def displayTapNodeData(data):
    return json.dumps(data, indent=2)


app.run_server(mode='inline')